In [46]:
import heapq
from collections import defaultdict, deque

In [2]:
with open("Data/Day_20_content.txt") as file:
    racetrack = [line.strip() for line in file.readlines()]

In [3]:
racetrack

['#############################################################################################################################################',
 '#...#.....###.....#.......................###...#...#...#####...#.....#...#...#...........#...#...#...###...#...###.......###...........#...#',
 '#.#.#.###.###.###.#.#####################.###.#.#.#.#.#.#####.#.#.###.#.#.#.#.#.#########.#.#.#.#.#.#.###.#.#.#.###.#####.###.#########.#.#.#',
 '#.#.#...#.....#...#...#...#.....#.........#...#.#.#...#...#...#.#...#...#.#.#.#.....#.....#.#...#...#.....#...#...#.#.....#...#.........#.#.#',
 '#.#.###.#######.#####.#.#.#.###.#.#########.###.#.#######.#.###.###.#####.#.#.#####.#.#####.#####################.#.#.#####.###.#########.#.#',
 '#.#...#.......#...###...#.#.#...#.#.....###...#.#.#...#...#...#.....#.....#.#.#...#.#.....#.#.....#...........#...#.#.#...#.#...#...###...#.#',
 '#.###.#######.###.#######.#.#.###.#.###.#####.#.#.#.#.#.#####.#######.#####.#.#.#.#.#####.#.#.###.#.#########.#.###.#.#.#.

## Part 1

In [4]:
# Parse racetrack to find start and end positions
grid = []
for r, row in enumerate(racetrack):
    grid_row = []
    
    for c, char in enumerate(row):
        
        if char == "S":
            start = (r, c)
            
        elif char == "E":
            end = (r, c)
            
        grid_row.append(char)
    
    grid.append(grid_row)

In [ ]:
["".join(row) for row in grid]

In [45]:
def heuristic(a, b): return abs(a[0] - b[0]) + abs(a[1] - b[1])

In [21]:
# Perform BFS to find the base amount of time to complete grid maze
def bfs(grid, start, end):
    
    rows, cols = len(grid), len(grid[0])
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    queue = deque([(start[0], start[1], 0)])  # row, col, time
    visited = defaultdict(lambda: float("inf"))  # (row, col) -> min_time
    visited[(start[0], start[1])] = 0
    
    while queue:
        r, c, time = queue.popleft()
        
        if (r, c) == end:
            return time
        
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            
            # Normal movement
            if 0 <= nr < rows and 0 <= nc < cols and grid[nr][nc] != "#":
                
                normal_time = time + 1
                
                if normal_time < visited[(nr, nc)]:
                    
                    visited[(nr, nc)] = normal_time
                    queue.append((nr, nc, normal_time))

In [47]:
def astar_with_cheats(grid, start, end):
    rows, cols = len(grid), len(grid[0])
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    open_set = []
    heapq.heappush(open_set, (0, start, 0, False, None, None))  # (priority, row, col), time, used_cheat, cheat_start, cheat_end
    visited = defaultdict(lambda: float("inf"))  # (row, col, used_cheat) -> min_time
    visited[(start[0], start[1], False)] = 0
    cheat_paths = []
    
    while open_set:
        priority, (r, c), time, used_cheat, cheat_start, cheat_end = heapq.heappop(open_set)
        
        if (r, c) == end and used_cheat:
            cheat_paths.append((cheat_start, cheat_end, time))
            continue
        
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            
            # Normal movement
            if 0 <= nr < rows and 0 <= nc < cols and grid[nr][nc] != "#":
                
                normal_time = time + 1
                
                if normal_time < visited[(nr, nc, used_cheat)]:
                    
                    visited[(nr, nc, used_cheat)] = normal_time
                    new_priority = normal_time + heuristic((nr, nc), end)
                    heapq.heappush(open_set, (new_priority, (nr, nc), normal_time,
                                              used_cheat, cheat_start, cheat_end))
                    
            # Cheat movement
            if not used_cheat:
                for i in range(1, 3):  # Cheat for 1 or 2 steps
                
                    cr, cc = r + dr * i, c + dc * i
                    
                    if 0 <= cr < rows and 0 <= cc < cols:
                        if grid[cr][cc] != "#":
                            
                            cheated_time = time + i
                            
                            if cheated_time < visited[(cr, cc, True)]:
                                    
                                visited[(cr, cc, True)] = cheated_time
                                new_cheat_start = (r, c)
                                new_cheat_end = (cr, cc)
                                new_priority = cheated_time + heuristic((cr, cc), end)
                                heapq.heappush(open_set, (new_priority, (cr, cc), cheated_time,
                                                          True, new_cheat_start, new_cheat_end))
                                
                        else:
                            continue
                    
                    #else:
                    #    break
                
    return cheat_paths

In [48]:
normal_time = bfs(grid, start, end)
cheat_paths = astar_with_cheats(grid, start, end)

count = 0
for cheat_start, cheat_end, cheat_time in cheat_paths:
    total_cheat_time = bfs(grid, cheat_end, end)
    if normal_time - (cheat_time + total_cheat_time) >= 100:
        count += 1
        
count

1